# XML로 가져오기
* xml: html처럼 태그 기반으로 자료를 저장한 포멧
* xml parser를 통해서 str을 xml로 변환하는 작업이 필요
* xml로 변환이 되면 태그 기반으로 자료를 찾아서 정리
* 태그에서 자료를 추출할 때는 beautifulsoup이라는 라이브러리를 이용

# BeautifulSoup
[https://www.crummy.com/software/BeautifulSoup/bs4/doc/](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
```python
!pip install beautifulsoup4 lxml
from bs4 import BeautifulSoup as bs
```
* find, find_all 함수: xml, html에서 태그 기반으로 내용을 찾음
* select, select_one 함수: xml, html에서 css selector기반으로 내용을 찾음
* find_all, select는 해당 태그나 css selector를 가진 부분을 모두 찾아서 list로 반환
* find, select_one은 여러 태그나 css selector 중에서 가장 먼저 나오는 태그/selector를 한 개만 찾아줌
* 찾아온 태그 .name: 태그 이름 반환
* 찾아온 태그 .get_text(): 태그 안쪽의 텍스트 반환

In [2]:
# !pip install beautifulsoup4 lxml

In [5]:
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
</body>
</html>
"""

In [4]:
html_doc[:4]

'<htm'

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

print(soup.prettify())

<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
 <body>
  <p class="title">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    Elsie
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>



In [8]:
soup.find("title").get_text()

"The Dormouse's story"

In [12]:
for link in soup.find_all("a"):
    print(link.get_text(), link['href'])

Elsie http://example.com/elsie
Lacie http://example.com/lacie
Tillie http://example.com/tillie


* find, find_all  태그 기반으로 찾기
  * find 는 html이나 xml 에서 가장 위에 있는 태그 1개만 찾아줌
  * find_all은 html 내부에 있는 모든 태그를 찾아서 list로 만들어줌

In [13]:
soup.find("b")

<b>The Dormouse's story</b>

In [27]:
soup.find("a")

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [18]:
# find_all 은 태그를 찾아서 list로 반환, for 반복문 필요
soup.find_all("a")[::-1]

[<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

In [25]:
soup.find_all("a")[0]['href']

'http://example.com/elsie'

* find로 찾은 태그의 이름 출력 .name  => dict의 key 출력
* find로 찾은 태그 안의 내용 출력 .get_text() => dict의 value 출력

In [29]:
# html에 있는 모든 태그의 이름 출력
for tag in soup.find_all():
    print(tag.name, tag.get_text())

html The Dormouse's story

The Dormouse's story
Once upon a time there were three little sisters; and their names were
Elsie,
Lacie and
Tillie;
and they lived at the bottom of a well.
...


head The Dormouse's story
title The Dormouse's story
body 
The Dormouse's story
Once upon a time there were three little sisters; and their names were
Elsie,
Lacie and
Tillie;
and they lived at the bottom of a well.
...

p The Dormouse's story
b The Dormouse's story
p Once upon a time there were three little sisters; and their names were
Elsie,
Lacie and
Tillie;
and they lived at the bottom of a well.
a Elsie
a Lacie
a Tillie
p ...


* select, selet_one
  * select: html이나 xml 안에서 tag나 css selector를 모두 찾아서 list로 만들어줌
  * select_one: html이나 xml 안에서 tag나 css selector를 찾아서 가장 위의 1개만 출력

In [34]:
soup.select_one(".sister").get_text()

'Elsie'

In [35]:
soup.select(".sister")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

# 소상공인 상가(상권) 정보를 xml로 가져와서 csv로 저장하기

In [36]:
import os
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
url = "https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong"
payload = dict(divId="indsSclsCd", key="I21006", numOfRows=2, pageNo=1, type="xml", serviceKey=os.getenv("service_key"))
r = requests.get(url, params=payload)
print(r.url)
print(r.status_code)
soup = bs(r.content, "xml")
soup

https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=2&pageNo=1&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200


<?xml version="1.0" encoding="utf-8"?>
<response>
<header>
<description>소상공인시장진흥공단 주요상권내 상가업소정보</description>
<columns>상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,PNU코드,대지구분코드,대지구분명,지번본번지,지번부번지,지번주소,도로명코드,도로명,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도</columns>
<stdrYm>202509</stdrYm>
<resultCode>00</resultCode>
<resultMsg>NORMAL SERVICE</resultMsg>
</header>
<body>
<items>
<item>
<bizesId>MA010120220800003146</bizesId>
<bizesNm>깐부치킨</bizesNm>
<brchNm/>
<indsLclsCd>I2</indsLclsCd>
<indsLclsNm>음식</indsLclsNm>
<indsMclsCd>I210</indsMclsCd>
<indsMclsNm>기타 간이</indsMclsNm>
<indsSclsCd>I21006</indsSclsCd>
<indsSclsNm>치킨</indsSclsNm>
<ksicCd>I56194</ksicCd>
<ksicNm>김밥 및 기타 간이 음식점업</ksicNm>
<ctprvnCd>41</ctprvnCd>
<ctprvnNm>경기도</ctprvnNm>
<signguCd>41210</signguCd>
<signguNm>광명시</signguNm>
<adongCd>41210640</adongCd>
<adongNm>소하1동</adongNm>
<ldongCd>4121010400</ldongCd>
<ldongNm>소하동</ldongNm>

In [45]:
soup.find('columns').get_text()

'상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,PNU코드,대지구분코드,대지구분명,지번본번지,지번부번지,지번주소,도로명코드,도로명,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도'

In [53]:
soup.find_all('item')

[<item>
 <bizesId>MA010120220800003146</bizesId>
 <bizesNm>깐부치킨</bizesNm>
 <brchNm/>
 <indsLclsCd>I2</indsLclsCd>
 <indsLclsNm>음식</indsLclsNm>
 <indsMclsCd>I210</indsMclsCd>
 <indsMclsNm>기타 간이</indsMclsNm>
 <indsSclsCd>I21006</indsSclsCd>
 <indsSclsNm>치킨</indsSclsNm>
 <ksicCd>I56194</ksicCd>
 <ksicNm>김밥 및 기타 간이 음식점업</ksicNm>
 <ctprvnCd>41</ctprvnCd>
 <ctprvnNm>경기도</ctprvnNm>
 <signguCd>41210</signguCd>
 <signguNm>광명시</signguNm>
 <adongCd>41210640</adongCd>
 <adongNm>소하1동</adongNm>
 <ldongCd>4121010400</ldongCd>
 <ldongNm>소하동</ldongNm>
 <lnoCd>4121010400113390001</lnoCd>
 <plotSctCd>1</plotSctCd>
 <plotSctNm>대지</plotSctNm>
 <lnoMnno>1339</lnoMnno>
 <lnoSlno>1</lnoSlno>
 <lnoAdr>경기도 광명시 소하동 1339-1</lnoAdr>
 <rdnmCd>412103187036</rdnmCd>
 <rdnm>경기도 광명시 소하로</rdnm>
 <bldMnno>81</bldMnno>
 <bldSlno/>
 <bldMngNo>4121010400113390001000001</bldMngNo>
 <bldNm>아이플렉스상가</bldNm>
 <rdnmAdr>경기도 광명시 소하로 81</rdnmAdr>
 <oldZipcd>423051</oldZipcd>
 <newZipcd>14316</newZipcd>
 <dongNo/>
 <flrNo>1</flrNo>
 

In [61]:
result = {}
for item in soup.find_all('item'):
    for tags in item.find_all(recursive=False):
        result.setdefault(tags.name, []).append(tags.get_text())
df = pd.DataFrame(result)
df

,bizesId,bizesNm,brchNm,indsLclsCd,indsLclsNm,indsMclsCd,indsMclsNm,indsSclsCd,indsSclsNm,ksicCd,...,bldMngNo,bldNm,rdnmAdr,oldZipcd,newZipcd,dongNo,flrNo,hoNo,lon,lat
0,MA010120220800003146,깐부치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56194,...,4121010400113390001000001,아이플렉스상가,경기도 광명시 소하로 81,423051,14316,,1,,126.884017082996,37.4476383411883
1,MA010120220800001311,페리카나,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1111013700100490000014132,,서울특별시 종로구 삼일대로 434-1,110320,03133,,,,126.988181843857,37.573537956838


In [72]:
int(soup.find("totalCount").get_text()) // 1000 

38

# xml에서 데이터추출하기

In [68]:
url = "https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong"
payload = dict(divId="indsSclsCd", key="I21006", numOfRows=1000, pageNo=1, type="xml", serviceKey=os.getenv("service_key"))
r = requests.get(url, params=payload)
print(r.url)
print(r.status_code)
soup = bs(r.content, "xml")
result = {}
for item in soup.find_all('item'):
    for tags in item.find_all(recursive=False):
        result.setdefault(tags.name, []).append(tags.get_text())
df = pd.DataFrame(result)
df

https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=1000&pageNo=1&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200


,bizesId,bizesNm,brchNm,indsLclsCd,indsLclsNm,indsMclsCd,indsMclsNm,indsSclsCd,indsSclsNm,ksicCd,...,bldMngNo,bldNm,rdnmAdr,oldZipcd,newZipcd,dongNo,flrNo,hoNo,lon,lat
0,MA010120220800497493,라피노치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1150010800100340040006076,,서울특별시 강서구 공항대로8길 11,157812,07624,,1,,126.812379770359,37.5605683033234
1,MA010120220800507495,썬더치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1150010300103530093014082,,서울특별시 강서구 강서로17다길 6,157882,07769,,,,126.842340698494,37.5327892916672
2,MA010120220800416449,통다리치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,2818510200103560001043236,,인천광역시 연수구 선학로68번길 7-5,406090,21910,,1,,126.701486008637,37.4239820387925
3,MA010120220800645292,누나홀닭평내,호평점,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,4136010100101650004015245,,경기도 남양주시 호평로67번길 1,472120,12147,,,,127.247323080721,37.6563143814352
4,MA010120220800103915,금강두마리치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,4423025032102490002051821,금강빌리지임대,충청남도 논산시 강경읍 동안로 96,320904,32942,,,,127.022380009199,36.1514064114243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,MA010120220800757579,굽네치킨월산점,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,2914011900107240025022670,,광주광역시 서구 화정로 258-1,502841,62047,,1,,126.88507944575,35.1477265797091
996,MA010120220800102679,네네치킨성산점,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,5013025924111320008000002,,제주특별자치도 서귀포시 성산읍 고성동서로56번길 18,699901,63640,,,,126.914698569169,33.4497918215461
997,MA010120220800560972,교촌치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1138011000102090018011448,,서울특별시 은평구 증산서길 65,122940,03493,,1,,126.905522506506,37.5826368211442
998,MA010120220800755551,"오태식해바라기치킨진월,주월점",,I2,음식,I210,기타 간이,I21006,치킨,I56192,...,2915510800103300017015306,,광주광역시 남구 광복마을5길 16,503841,61732,,1,,126.895239002778,35.1239307994294


# 전체 데이터 개수로 page수 계산 후 모든 데이터 수집
* 컬럼명을 한국어 컬럼명으로 만들기

In [86]:
# len(soup.find("columns").get_text().split(","))

In [87]:
# kor_cols = soup.find("columns").get_text().split(",")
# kor_cols

In [88]:
# len([tag.name for tag in soup.find("item").find_all(recursive=False)])

In [94]:
if int(soup.find("totalCount").get_text()) % 1000 == 0:
    total_pages = int(soup.find("totalCount").get_text()) // 1000
else:
    total_pages = int(soup.find("totalCount").get_text()) // 1000 + 1
total_pages

39

In [95]:
result = {}
page = 1
total_pages = 1

while True:
    
    # 종료 조건
    if page > total_pages:
        break
    
    url = "https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong"
    payload = dict(divId="indsSclsCd", key="I21006", numOfRows=1000, pageNo=page, type="xml", serviceKey=os.getenv("service_key"))
    r = requests.get(url, params=payload)
    print(r.url)
    print(r.status_code)
    soup = bs(r.content, "xml")

    # 한국어 컬럼 추출하기
    kor_cols = soup.find("columns").get_text().split(",")

    # 전체 페이지 구하기
    if total_pages == 1:
        if int(soup.find("totalCount").get_text()) % 1000 == 0:
            total_pages = int(soup.find("totalCount").get_text()) // 1000
        else:
            total_pages = int(soup.find("totalCount").get_text()) // 1000 + 1

    # 데이터 추출해서 result에 모으기
    for item in soup.find_all('item'):
        for kor_col, tags in zip(kor_cols, item.find_all(recursive=False)):
            result.setdefault(kor_col, []).append(tags.get_text())
       
    page += 1

df = pd.DataFrame(result)
df

https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=1000&pageNo=1&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200
https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=1000&pageNo=2&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200
https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=1000&pageNo=3&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200
https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=1000&pageNo=4&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200
https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong

https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=1000&pageNo=36&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200
https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=1000&pageNo=37&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200
https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=1000&pageNo=38&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200
https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong?divId=indsSclsCd&key=I21006&numOfRows=1000&pageNo=39&type=xml&serviceKey=8Ym5dhmVJdr12XzGnyYrQjSBS1QuRBYK8yHfx65JCl4vACM9uLKo8fxCVOFJkPB71llD7F2rOEROZnHgNGoj3A%3D%3D
200


,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA010120220800497493,라피노치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1150010800100340040006076,,서울특별시 강서구 공항대로8길 11,157812,07624,,1,,126.812379770359,37.5605683033234
1,MA010120220800507495,썬더치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,1150010300103530093014082,,서울특별시 강서구 강서로17다길 6,157882,07769,,,,126.842340698494,37.5327892916672
2,MA010120220800416449,통다리치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,2818510200103560001043236,,인천광역시 연수구 선학로68번길 7-5,406090,21910,,1,,126.701486008637,37.4239820387925
3,MA010120220800645292,누나홀닭평내,호평점,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,4136010100101650004015245,,경기도 남양주시 호평로67번길 1,472120,12147,,,,127.247323080721,37.6563143814352
4,MA010120220800103915,금강두마리치킨,,I2,음식,I210,기타 간이,I21006,치킨,I56193,...,4423025032102490002051821,금강빌리지임대,충청남도 논산시 강경읍 동안로 96,320904,32942,,,,127.022380009199,36.1514064114243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38395,MA0106202507A1225972,노랑통닭 미사역점,,I2,음식,I210,기타 간이,I21006,치킨,I56111,...,4145010900108300061000001,,경기도 하남시 미사강변중앙로 157,465150,12913,,,,127.190744125788,37.5606083475736
38396,MA0106202507A2084239,가마치통닭 용인동백점,,I2,음식,I210,기타 간이,I21006,치킨,I56111,...,4146311600108380000000766,,경기도 용인시 기흥구 동백5로 22,446766,17006,,,,127.15128147073,37.275961909971
38397,MA0106202507A0834201,솜리치킨에코시티점,,I2,음식,I210,기타 간이,I21006,치킨,I56111,...,4511312200113270015000001,,전북특별자치도 전주시 덕진구 세병2길 19,561302,55149,,,,127.126156942737,35.8729156645355
38398,MA0106202507A0594715,낭만장작창원팔용점,,I2,음식,I210,기타 간이,I21006,치킨,I56111,...,4812112900101740007018198,,경상남도 창원시 의창구 남산로27번길 7,641851,51368,,,,128.613482429916,35.2594256051816


# 모든 카테고리(업종)데이터 수집하고 업종별로 저장하기

In [96]:
upjong = pd.read_excel("./상권상가API_매뉴얼/소상공인시장진흥공단_상가(상권)정보_업종분류(2302)_및_연계표_v1.xlsx", header=1)
upjong

,대분류코드,대분류명,중분류코드,중분류명,소분류코드,소분류명
0,G2,소매업,G202,자동차 부품 및 내장품 소매업,G20201,타이어 소매업
1,G2,소매업,G202,자동차 부품 및 내장품 소매업,G20202,자동차 부품 소매업
2,G2,소매업,G203,모터사이클 및 부품 소매업,G20301,모터사이클 및 부품 소매업
3,G2,소매업,G204,종합 소매업,G20404,슈퍼마켓
4,G2,소매업,G204,종합 소매업,G20405,편의점
...,...,...,...,...,...,...
242,S2,수리 및 개인 서비스업,S209,세탁업,S20902,셀프 빨래방
243,S2,수리 및 개인 서비스업,S210,장례식장 및 관련 서비스업,S21001,장례식장
244,S2,수리 및 개인 서비스업,S210,장례식장 및 관련 서비스업,S21002,화장터/묘지/납골당
245,S2,수리 및 개인 서비스업,S211,기타 개인서비스,S21101,예식장업


In [100]:
codes = upjong['소분류코드']
names = upjong['소분류명']

In [104]:
for idx, (code, name) in enumerate(zip(codes[:3], names)):
#     print(code, name)
    result = {}
    page = 1
    total_pages = 1

    while True:

        # 종료 조건
        if page > total_pages:
            break

        url = "https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong"
        payload = dict(divId="indsSclsCd", key=code, numOfRows=1000, pageNo=page, type="xml", serviceKey=os.getenv("service_key"))
        r = requests.get(url, params=payload)
#         print(r.url)
#         print(r.status_code)
        soup = bs(r.content, "xml")

        # 한국어 컬럼 추출하기
        kor_cols = soup.find("columns").get_text().split(",")

        # 전체 페이지 구하기
        if total_pages == 1:
            if int(soup.find("totalCount").get_text()) % 1000 == 0:
                total_pages = int(soup.find("totalCount").get_text()) // 1000
            else:
                total_pages = int(soup.find("totalCount").get_text()) // 1000 + 1

        print(f"{idx}/{len(names)-1} 카테고리 데이터 {page}/{total_pages}", end="\r")
        
        # 데이터 추출해서 result에 모으기
        for item in soup.find_all('item'):
            for kor_col, tags in zip(kor_cols, item.find_all(recursive=False)):
                result.setdefault(kor_col, []).append(tags.get_text())

        page += 1
        
        time.sleep(0.3)
    # 폴더 자동으로 생성하기
    dir_name = "shops_data_xml"
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
                
    df = pd.DataFrame(result)
    df.to_csv(f"./{dir_name}/{name}_데이터.csv", encoding="utf-8-sig")